In [ ]:
import gc
import multiprocessing
from multiprocessing.pool import ThreadPool
import math
import os
import pathlib
import pickle
import random
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import dask
import dask.multiprocessing


from tqdm import tqdm_notebook

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, train_test_split

import lightgbm as lgb
import xgboost as xgb

import keras
from keras.models import Sequential
from keras.layers import Dense, CuDNNGRU
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint
from tensorflow import set_random_seed

In [ ]:
%matplotlib inline

In [ ]:
dask.config.set(scheduler='processes')
dask.set_options( pool=ThreadPool(10) )

In [ ]:
def add_features_dask_parralel(
#def add_features_parallel(
        df,
        input_first_index=None,
        input_last_index=None,
        sample_size=150000,
        holdout_size=50000,
        smootch_windows_size = (3, 5, 7)
    ):
    if input_first_index == None or input_last_index == None:
        input_first_index = df.index.min()
        input_last_index = df.index.max() + 1
        
    
    sample_indexes = random.sample(range(input_first_index, input_last_index), sample_size)
    sample_indexes.sort()
    
    smootch_feature_names = ['smootch_mean_ws_{}'.format(window_size) for window_size in smootch_windows_size]
    acoustic_data_series = df['acoustic_data']
    full_data_indexes = tuple(acoustic_data_series.index.tolist())

    sample_df = df.iloc[sample_indexes]

    sample_df.reset_index(inplace=True)
    sample_df.drop(columns=['index'], inplace=True)

    output_first_index = 0
    output_last_index = len(sample_df) - 1
    
    begin_indexes_set = set()
    end_indexes_set = set()
    
    start_time = time.time()
   
    sample_df_len = sample_df.shape[0]
    @dask.delayed
    def create_features():
        for window_size, feature_name in zip(smootch_windows_size, smootch_feature_names):

            feature_values_list = list(range(sample_size))

            half_window_size = window_size // 2

            sample_begin_indexes = sample_indexes[:half_window_size]
            full_data_begin_indexes = set(df.index[sample_begin_indexes].tolist())
            min_full_data_index = min(full_data_indexes)
        
            in_window_full_data_begin_indexes = set(range(input_first_index, input_first_index + half_window_size))              
            in_window_begin_indexes = full_data_begin_indexes.intersection(
                in_window_full_data_begin_indexes
            )
        
            sample_end_indexes = sample_indexes[-half_window_size:]
            full_data_end_indexes = set(df.index[sample_end_indexes].tolist())
            max_full_data_index = max(full_data_end_indexes) + 1
        
            in_window_full_data_end_indexes = set(range(input_last_index - half_window_size, input_last_index))        
            in_window_end_indexes = full_data_end_indexes.intersection(
                in_window_full_data_end_indexes
            )
            if in_window_begin_indexes:
                begin_indexes_set = begin_indexes_set.union(in_window_begin_indexes)
                for i, b_idx in enumerate(sorted(tuple(in_window_begin_indexes))):
                    value = sample_df.iloc[i]['acoustic_data']
                    temp = acoustic_data_series.iloc[input_first_index:input_first_index + window_size].mean()
                    value = value - temp
                    feature_values_list[output_first_index + i] = value
                
            if in_window_end_indexes:
                end_indexes_set = end_indexes_set.union(in_window_end_indexes)
                for i, e_idx in enumerate(sorted(tuple(in_window_end_indexes))):
                    value = sample_df.iloc[output_last_index - i]['acoustic_data']
                    temp = acoustic_data_series.iloc[input_last_index - window_size:].mean()
                    value = value - temp
                    feature_values_list[output_last_index - i] = value
                
            first_regular_idx = len(begin_indexes_set)
            last_regular_idx = sample_df_len - len(end_indexes_set)
            for i in range(first_regular_idx, last_regular_idx):
                sample_idx = sample_indexes[i]
                feature_values_list[i] = acoustic_data_series.iloc[
                    sample_idx - half_window_size:sample_idx + half_window_size
                ].mean()
            sample_df[feature_name] = feature_values_list
        return sample_df
        
    holdout_df = None
    if holdout_size > 0:
        holdout_indexes = np.random.randint(0, sample_df.shape[0], holdout_size)
        holdout_df = sample_df.iloc[holdout_indexes]
        holdout_df.reset_index(inplace=True)
        holdout_df.drop(columns=['index'], inplace=True)
        train_indexes = sorted(tuple(set(sample_df.index).difference(set(holdout_indexes))))
        sample_df = sample_df.iloc[train_indexes]
        sample_df.reset_index(inplace=True)
        sample_df.drop(columns=['index'], inplace=True)
    print("Full calculation feature value time (with slicing) {} min:".format((time.time() - start_time) / 60))
    return sample_df, holdout_df

In [ ]:
add_features = add_features_dask_parralel

In [ ]:
def extract_features(z):
    print()
    print("extract_features, z.shape:", z.shape)
    #print()
    return np.c_[z.mean(axis=1),
                 np.transpose(np.percentile(np.abs(z), q=[0, 50, 75, 100], axis=1)),
                 z.std(axis=1)]

In [ ]:
def create_X(x, last_index=None, n_steps=150, step_length=1000):
    if last_index == None:
        last_index=len(x)
    
    print("\n")
    print("createX, x.shape:", x.shape)
    print("createX, last_index:", last_index)
    print("createX, n_steps:", n_steps)
    print("createX, step_length:", step_length)
    assert last_index - n_steps * step_length >= 0

    # Reshaping and approximate standardization with mean 5 and std 3.
    # ORIGINAL: I changed this becuase I got an No OpKernel was registered to support Op 'CudnnRNN' error
    #temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1) - 5 ) / 3
    # MY CHANGE: This doesn't fix things, I get the same errors
    #temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1).astype(np.float32) - 5 ) / 3
    temp = x[(last_index - n_steps * step_length):last_index]
    print("createX, temp.shape before reshape:", temp.shape)
    temp = temp.reshape(n_steps, -1).astype(np.float32)
    print("create_X, temp.shape after reshape:", temp.shape)
    temp = (temp - 5) / 3
    
    # Extracts features of sequences of full length 1000, of the last 100 values and finally also 
    # of the last 10 observations.
    print("createX, extract_features(temp).shape:", extract_features(temp).shape)
    print("createX, extract_features(temp[:, -step_length // 10:]).shape", extract_features(temp[:, -step_length // 10:]).shape)
    print("createX, extract_features(temp[:, -step_length // 100:]).shape", extract_features(temp[:, -step_length // 100:]).shape)
    print()
    #result = np.c_[extract_features(temp),
    #             extract_features(temp[:, -step_length // 10:]),
    #             extract_features(temp[:, -step_length // 100:])]
    result = np.c_[
        temp,
        temp[:, -step_length // 10:],
        temp[:, -step_length // 100:]
    ]
    print("createX, result shape:", result.shape)
    print()
    '''
    return np.c_[extract_features(temp),
                 extract_features(temp[:, -step_length // 10:]),
                 extract_features(temp[:, -step_length // 100:])]
    '''
    return result

In [ ]:
# The generator endlessly selects "batch_size" ending positions of sub-time series. For each ending position,
# the "time_to_failure" serves as target, while the features are created by the function "create_X".
#def generator(data, min_index=0, max_index=None, batch_size=16, n_steps=150, step_length=1000):
def generator(data, y, rows, batch_size=16, n_steps=150, step_length=1000):
    #if max_index is None:
    #    max_index = len(data) - 1
    print("\n")   
    print("generator, data.shape:", data.shape)
    
    '''
    while True:
        # Pick indices of ending positions
        #rows = np.random.randint(min_index + n_steps * step_length, max_index, size=batch_size)
        print("generator, type(rows):", type(rows))
        print("generator, rows.shape:", rows.shape)
        print("generator rows:\n", rows)
         
        # Initialize feature matrices and targets
        samples = np.zeros((batch_size, n_steps, n_features))
        targets = np.zeros(batch_size, )
        print("generator, samples.shape:", samples.shape)
        print("generator, targets.shape:", targets.shape)
        
        for j, row in enumerate(rows):
            print("generator j: {}, row: {}".format(j, row))
            samples[j] = create_X(data[:, 0], last_index=row, n_steps=n_steps, step_length=step_length)
            print("generator, samples[{}].shape:".format(j), samples[j].shape)
            print("generator, row - 1:", row - 1)
            targets[j] = data[row - 1, 1]
            print("generator, targets[{}].shape:".format(j), targets[j].shape)
            print()
        yield samples, targets
    '''
        
    while True:
        
        # Pick indices of ending positions
        rows = np.random.randint(min_index + n_steps * step_length, max_index, size=batch_size)
         
        # Initialize feature matrices and targets
        samples = np.zeros((batch_size, n_steps, n_features))
        targets = np.zeros(batch_size, )
        
        for j, row in enumerate(rows):
            samples[j] = create_X(data, last_index=row, n_steps=n_steps, step_length=step_length)
            #targets[j] = data[row - 1, 1]
            targets[j] = y[row - 1]
            #temp = x[(last_index - n_steps * step_length):last_index]
            #print("createX, temp.shape before reshape:", temp.shape)
            #temp = temp.reshape(n_steps, -1).astype(np.float32)
            #print("create_X, temp.shape after reshape:", temp.shape)
            #temp = (temp - 5) / 3
        yield samples, targets

In [ ]:
cb = [ModelCheckpoint("model.hdf5", save_best_only=True, period=3)]

model = Sequential()
model.add(CuDNNGRU(48, input_shape=(None, n_features)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

model.summary()

In [ ]:
model.compile(optimizer=adam(lr=0.0005), loss="mae")

history = model.fit_generator(train_gen,
                              steps_per_epoch=1000,
                              epochs=30,
                              verbose=0,
                              callbacks=cb,
                              validation_data=valid_gen,
                              validation_steps=200)

In [ ]:
earthquake_margin_indexes =[
    5656573,
    50085877,
    104677355,
    138772452,
    187641819,
    218652629,
    245829584,
    307838916,
    338276286,
    375377847,
    419368879,
    461811622,
    495800224,
    528777114,
    585568143,
    621985672
]

In [ ]:
earthquakes_length = [earthquake_margin_indexes[i + 1] - earthquake_margin_indexes[i] for i in range(len(earthquake_margin_indexes) - 1)]

In [ ]:
earthquakes_length

In [ ]:
complete_earthquakes_length = earthquakes_length[:-1]

In [ ]:
#complete_earthquaces_length = complete_earthquaces_length[:-1]

In [ ]:
complete_earthquakes_length

In [ ]:
#%%time
#not_seen_data_df = pd.read_csv(
#    '../input/train/train.csv',
#    #nrows=100000000,
#    names=['acoustic_data', 'time_to_failure'],
#    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32},
#    skiprows=1,
#    nrows=5656572
#)

In [ ]:
#%%time
#featured_not_seen_data_df, _ = add_features(
#    not_seen_data_df,
#    sample_size=not_seen_data_df.shape[0] // 20,
#    holdout_size=0
#)

In [ ]:
#y_not_seen_data = featured_not_seen_data_df['time_to_failure']
#featured_not_seen_data_no_time_df = featured_not_seen_data_df[featured_not_seen_data_df.columns.drop('time_to_failure')]

In [ ]:
#float_data = pd.read_csv("../input/train/train.csv", dtype={"acoustic_data": np.float32, "time_to_failure": np.float32}).values

In [ ]:
%%time
#        earthquake_df = pd.read_csv(
#                '../input/train/train.csv',
#                #nrows=100000000,
#                names=['acoustic_data', 'time_to_failure'],
#                dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32},
#                skiprows=earthquake_margin_indexes[i],
#                nrows=complete_earthquakes_length[i]
#            )

train_df = pd.read_csv('../input/train/train.csv', dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})

In [ ]:
#        if not sample_size:
#            sample_size = complete_earthquakes_length[i] // 10
#        if not holdout_size:
#            holdout_size = complete_earthquakes_length[i] // 50
sample_size = train_df.shape[0] // 10
holdout_size = train_df.shape[0] // 50
%time
featured_train_df, featured_holdout_df = add_features(
                train_df,
                sample_size=sample_size,
                holdout_size=holdout_size
            )

In [ ]:
X_all = featured_train_df[featured_train_df.columns.drop('time_to_failure')]
y_all = featured_train_df['time_to_failure']

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_all, y_all, test_size=0.2, random_state=0)

In [ ]:
# Query "create_X" to figure out the number of features
#n_features = create_X(float_data[0:150000]).shape[1]
n_features = create_X(train_df[0:150000]).shape[1]
print("Our RNN is based on %i features"% n_features)

In [ ]:
batch_size = 32

# Position of second (of 16) earthquake. Used to have a clean split
# between train and validation
second_earthquake = 50085877
#float_data[second_earthquake, 1]
train_df[second_earthquake, 1]

# Initialize generators
# train_gen = generator(float_data, batch_size=batch_size) # Use this for better score
# train_gen = generator(float_data, batch_size=batch_size, min_index=second_earthquake + 1)
# valid_gen = generator(float_data, batch_size=batch_size, max_index=second_earthquake)
train_gen = generator(train_df, batch_size=batch_size, min_index=second_earthquake + 1)
valid_gen = generator(train_df, batch_size=batch_size, max_index=second_earthquake)

In [ ]:
model.compile(optimizer=adam(lr=0.0005), loss="mae")

history = model.fit_generator(train_gen,
                              steps_per_epoch=1000,
                              epochs=30,
                              verbose=0,
                              callbacks=cb,
                              validation_data=valid_gen,
                              validation_steps=200)

In [ ]:
def perf_plot(history, what = 'loss'):
    x = history.history[what]
    val_x = history.history['val_' + what]
    epochs = np.asarray(history.epoch) + 1
    
    plt.plot(epochs, x, 'bo', label = "Training " + what)
    plt.plot(epochs, val_x, 'b', label = "Validation " + what)
    plt.title("Training and validation " + what)
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    return None

perf_plot(history)

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})

In [ ]:
for i, seg_id in enumerate(tqdm(submission.index)):
  #  print(i)
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    x = seg['acoustic_data'].values
    submission.time_to_failure[i] = model.predict(np.expand_dims(create_X(x), 0))

submission.head()

In [ ]:
submission.to_csv('submission.csv')